In [1]:
import nibabel as nib

img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz")
spacing = img.header.get_zooms()
print("Voxel spacing (x, y, z):", spacing)


Voxel spacing (x, y, z): (np.float32(0.9766), np.float32(0.9766), np.float32(1.25))


In [3]:
from scipy.ndimage import distance_transform_edt
import nibabel as nib
import numpy as np

dentition_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\Dentition.nii.gz")
mandible_img = nib.load(r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\mandible_manual.nii.gz")

dentition = dentition_img.get_fdata() > 0
mandible = mandible_img.get_fdata() > 0

# Get voxel spacing
spacing = dentition_img.header.get_zooms()[:3]  # (x, y, z)

# Compute distance map in millimeters
dist_map = distance_transform_edt(~mandible, sampling=spacing)

# Apply threshold in mm (e.g. 5 mm)
lower_mask = dentition & (dist_map < 5)
upper_mask = dentition & (~lower_mask)

# Save
affine = dentition_img.affine
nib.save(nib.Nifti1Image(lower_mask.astype(np.uint8), affine), r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_lower.nii.gz")
nib.save(nib.Nifti1Image(upper_mask.astype(np.uint8), affine), r"Z:\FacialDeformation_MPhys\rhabdo_data_proton\DICOMS\abby\UIDQQ0x1x1Hx7\comparison\dentition_upper.nii.gz")
